<a href="https://colab.research.google.com/github/brenoluca21/ProjectETLGenerativeAiDIO/blob/main/Projeto1ETLDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exercicio Prático para bootcamp DIO Santander Cientista de Dados

In [105]:
import pandas as pd

## Extract
Adaptado conforme o enunciado do exercício em: https://github.com/falvojr/santander-dev-week-2023/blob/master/SantanderDevWeek2023.ipynb

In [106]:
users = [
    {'id': 1, 'name': 'Breno',   'news': [], 'saldo': 4200.00, 'limite': 1500},
    {'id': 2, 'name': 'Nicolas', 'news': [], 'saldo': 1800.00, 'limite': 2000},
    {'id': 3, 'name': 'João',    'news': [], 'saldo': 120.50,  'limite': 5000},
    {'id': 4, 'name': 'Lucas',   'news': [], 'saldo': 950.00,  'limite': 2500}
]


In [107]:
df = pd.DataFrame(users)
print(df)

   id     name news   saldo  limite
0   1    Breno   []  4200.0    1500
1   2  Nicolas   []  1800.0    2000
2   3     João   []   120.5    5000
3   4    Lucas   []   950.0    2500


In [108]:
users_id = df['id'].tolist()
print(users_id)

[1, 2, 3, 4]


##Transform

In [109]:
from google.colab import userdata
import google.generativeai as genai

# pega a chave do Secret do Colab
api_key = userdata.get('API_KEY')

# configura o Gemini
genai.configure(api_key=api_key)

# cria o modelo
model = genai.GenerativeModel("gemini-flash-latest")


In [110]:
for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods:
    print(m.name)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-prev

In [111]:
def generate_ai_news(user):
    prompt = f"""
Você é um especialista em análise de crédito bancário.

Regra para decisão de limite:
- Se o saldo for maior ou igual ao dobro do limite atual → AUMENTAR limite
- Se o saldo for menor que metade do limite atual → DIMINUIR limite
- Caso contrário → MANTER limite

Dados do cliente:
Nome: {user['name']}
Saldo: R$ {user['saldo']:.2f}
Limite atual: R$ {user['limite']:.2f}

Crie uma mensagem curta para {user['name']}informando se o limite irá
AUMENTAR, DIMINUIR ou SE MANTER, com uma justificativa simples  (máx. 100 caracteres) .
"""

    response = model.generate_content(prompt)
    return response.text.strip()


In [112]:
import time
for user in users:
    news = generate_ai_news(user)
    print(news)

    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })

    time.sleep(10)


Breno, seu limite irá **AUMENTAR**, pois seu saldo de R$ 4200.00 é superior ao dobro do limite atual.
**Nicolas, seu limite será MANTIDO.** Seu saldo (R$ 1800) não atingiu os critérios de aumento (R$ 4000) ou redução (R$ 1000).
João, seu limite será **DIMINUÍDO**. O saldo atual é inferior à metade do limite estabelecido.
Lucas, seu limite irá **DIMINUIR**. Seu saldo (R$950) é inferior à metade do limite atual (R$1250).


##Load

In [113]:
df_final = pd.DataFrame(users)


In [114]:
df_final.to_csv("users_com_news.csv", index=False, encoding="utf-8-sig", sep=";")
